<a href="https://colab.research.google.com/github/facahto/Kaggle-competition-practice/blob/main/forecasting-store_sales/Stores_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, numpy as np
import torch
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as MSE, mean_absolute_error as MAE
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM


label_encoder = preprocessing.LabelEncoder()

Data

In [2]:
# Holiday Event
h_e = pd.read_csv('holidays_events.csv')
display(h_e.head(3))
print()
print(h_e.info())
#display(h_e[h_e.duplicated(keep=False)]) ----- NO DUPLICATED DATA

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB
None


In [3]:
# Oil
oil = pd.read_csv('oil.csv')
oil['dcoilwtico'].fillna(float(oil['dcoilwtico'].mean()), inplace=True)
display(oil.head(3))
print()
print(oil.info())
#display(oil[oil.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,date,dcoilwtico
0,2013-01-01,67.714366
1,2013-01-02,93.140000
2,2013-01-03,92.970000



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1218 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB
None


In [4]:
# Stores
stores = pd.read_csv('stores.csv')
display(stores.head(3))
print()
print(stores.info())
#display(stores[stores.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB
None


In [5]:
# Transactions
ts = pd.read_csv('transactions.csv')
display(ts)
print()
print(ts.info())
#display(ts[ts.duplicated(keep=False)])  ----- NO DUPLICATED DATA

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922
...,...,...,...
83483,2017-08-15,50,2804
83484,2017-08-15,51,1573
83485,2017-08-15,52,2255
83486,2017-08-15,53,932



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          83488 non-null  object
 1   store_nbr     83488 non-null  int64 
 2   transactions  83488 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.9+ MB
None


In [6]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
display(train.head())

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


In [7]:
# Preprocessing train data
train['family']= label_encoder.fit_transform(train['family'])
test['family']= label_encoder.fit_transform(test['family'])

In [8]:
#train = pd.merge(train, ts, how="left", on=["date", "store_nbr"])

In [9]:
nan     = train.isna().sum()
nan_row = train[train.isna().any(axis=1)]
print("Jumlah nilai NaN:", nan)

Jumlah nilai NaN: id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64


In [10]:
#train['transactions'].fillna(float(train['transactions'].mean()), inplace=True)


In [11]:
display(train)

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,0,0.000,0
1,1,2013-01-01,1,1,0.000,0
2,2,2013-01-01,1,2,0.000,0
3,3,2013-01-01,1,3,0.000,0
4,4,2013-01-01,1,4,0.000,0
...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,28,438.133,0
3000884,3000884,2017-08-15,9,29,154.553,1
3000885,3000885,2017-08-15,9,30,2419.729,148
3000886,3000886,2017-08-15,9,31,121.000,8


In [12]:
## predict oil price for test dataframe

B = pd.DataFrame(columns=['B'])
valB       = oil['dcoilwtico'][1:]
valB[1218] = 45
B['B']     = valB.values
oil = pd.concat([oil, B],axis = 1)

oil['date'] = pd.to_datetime(oil['date'])
oil['year'] = oil['date'].dt.year
oil['month'] = oil['date'].dt.month
oil['day'] = oil['date'].dt.day

scaler = MinMaxScaler()
oil[['dcoilwtico', 'B']] = scaler.fit_transform(oil[['dcoilwtico', 'B']])
display(oil)

,date,dcoilwtico,B,year,month,day
0,2013-01-01,0.491820,0.792965,2013,1,1
1,2013-01-02,0.792965,0.790951,2013,1,2
2,2013-01-03,0.790951,0.792728,2013,1,3
3,2013-01-04,0.792728,0.793675,2013,1,4
4,2013-01-07,0.793675,0.793794,2013,1,7
...,...,...,...,...,...,...
1213,2017-08-25,0.254175,0.239370,2017,8,25
1214,2017-08-28,0.239370,0.240081,2017,8,28
1215,2017-08-29,0.240081,0.234158,2017,8,29
1216,2017-08-30,0.234158,0.249556,2017,8,30


In [13]:
train_oil   = oil[:1000]
X_train_oil = train_oil.drop(['date','B'],axis=1)
y_train_oil = train_oil['B'].values.reshape(-1, 1)

test_oil   = oil[1000:]
X_test_oil = test_oil.drop(['date','B'],axis=1)
y_test_oil = test_oil['B'].values.reshape(-1, 1)


model_oil = LinearRegression()
model_oil.fit(X_train_oil, y_train_oil)

pred_oil = model_oil.predict(X_test_oil)

mae = MAE(y_test_oil, pred_oil)
rmse = np.sqrt(MSE(y_test_oil, pred_oil))
print("RMSE : % f" %(rmse))
print("MAE : % f" %(mae))

oil_list = []
a = oil.iloc[-1][['dcoilwtico', 'year','month','day']].values.reshape(1, -1)
for i in range(len(test)):
  k = model_oil.predict(a)
  oil_list.append(k[0][0])
  a = k
test['dcoilwtico'] = oil_list

RMSE :  0.075393
MAE :  0.061915


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: ignored

In [ ]:
print(a)

In [ ]:
train = pd.merge(train, oil, how="left", on="date")

In [ ]:
train['dcoilwtico'].fillna(float(train['dcoilwtico'].mean()), inplace=True)
nan     = train.isna().sum()
nan_row = train[train.isna().any(axis=1)]
print("Jumlah nilai NaN:", nan)

In [ ]:
train['date'] = pd.to_datetime(train['date'])
train['year'] = train['date'].dt.year
train['month'] = train['date'].dt.month
train['day'] = train['date'].dt.day

In [ ]:
display(train)

In [ ]:
train = train.drop('B',axis=1)

In [ ]:
display(train)

In [ ]:
test['date'] = pd.to_datetime(test['date'])
test['year'] = test['date'].dt.year
test['month'] = test['date'].dt.month
test['day'] = test['date'].dt.day
display(test)

In [ ]:
X = train.drop(['id','date','sales','year'],axis=1)
y = train['sales']

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()

# lakukan proses training / fitting data
model.fit(train_X, train_y)

pred_xgb_r = model.predict(test_X)

perbandingan_xgb = pd.DataFrame({
    'aktual': test_y,
    'prediksi': pred_xgb_r
})

mae = MAE(test_y, pred_xgb_r)
rmse = np.sqrt(MSE(test_y, pred_xgb_r))
print("RMSE : % f" %(rmse))
print("MAE : % f" %(mae))